In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from ttvfast import models
import ttvfast
from matplotlib.ticker import AutoMinorLocator
import emcee
import corner
from multiprocessing import Pool
from scipy.optimize import minimize
import os
os.environ['OMP_NUM_THREADS']='1'
os.nice(19)

#周期は別のコードで回帰分析したものを
#Tc0は一番最初のepochから
Tc0b = 1524.088335
Pb = 6.39804982e+00
Tc0c = 2232.168625
Pc = 1.88792444e+01

#データの読み込み→epoch, time, errをlistに格納
data_b = pd.read_csv("TTV_TOI-560b_2.csv", header=None)
epoch_b = data_b.iloc[:,0]
ob_b = list(data_b.iloc[:,1])
ob_err_b = list(data_b.iloc[:,2])
data_c = pd.read_csv("TTV_TOI-560c_2.csv", header=None)
epoch_c = data_c.iloc[:,0]
ob_c = list(data_c.iloc[:,1])
ob_err_c = list(data_c.iloc[:,2])

#観測期間を指定
Tstart = np.min(np.array((ob_b[0], ob_c[0]))) - 0.5
Tend = np.max(np.array((ob_b[-1], ob_c[-1]))) + 0.5

#Tstartから最初の観測までは何周期たっているか（その切り捨て）
epc0 = np.floor((Tc0c-Tstart)/Pc)
epb0 = np.floor((Tc0b-Tstart)/Pb)

#Tstartから任意の観測までは何周期たっているか（その切り捨てすなわちepoch）
#ttvfastは0スタート
epc = np.floor((ob_c-Tstart)/Pc)
epb = np.floor((ob_b-Tstart)/Pb)

#pを渡してttvfastから得られる配列を渡す関数を定義
def calc_ttvs_2pl(p):
    mp1 = p[0]
    sqecosw1 = p[1]
    sqesinw1 = p[2]
    wpM1 = p[3] # w + M, radians,0~2*pi
    P1 = p[4]

    mp2 = p[5]
    sqecosw2 = p[6]
    sqesinw2 = p[7]
    wpM2 = p[8] # w + M, radians,0~2*pi
    P2 = p[9]
    
    #軌道要素に戻す
    e1 = sqecosw1**2 + sqesinw1**2
    e2 = sqecosw2**2 + sqesinw2**2
    w1 = np.arctan2(sqesinw1, sqecosw1) #radian,-pi~pi←ただしい
    w2 = np.arctan2(sqesinw2, sqecosw2) #radian,-pi~pi←ただしい
    M1 = wpM1 - w1 #radian
    M2 = wpM2 - w2 #radian
    
    #Mを0以上2pi以下にしよう←ほんと？
    #wとMは何以上何以下にすれば良いか
    if M1<0:
        M1+=2*np.pi
    if M1>2*np.pi:
        M1-=2*np.pi
    if M2<0:
        M2+=2*np.pi
    if M2>2*np.pi:
        M2-=2*np.pi
        
    
    gravity = 0.000295994511   # AU^3/day^2/M_sun
    stellar_mass = 0.73    # M_sun

    #多分これでいい
    planet1 = models.Planet(
        mass=mp1 / 332946.,   # M_sun
        period=P1,              # days
        eccentricity=e1,
        inclination=90,         # degrees
        longnode=180,           # degrees
        argument=np.rad2deg(w1),       
        mean_anomaly= np.rad2deg(M1),     
    )

    planet2 = models.Planet(
        mass=mp2 / 332946.,
        period=P2,
        eccentricity=e2,
        inclination=90,
        longnode=180,
        argument=np.rad2deg(w2),
        mean_anomaly= np.rad2deg(M2),
    )
    
    planets = [planet1, planet2]
    dt = 0.1

    return ttvfast.ttvfast(planets, stellar_mass, Tstart, dt, Tend)

In [ ]:
#尤度関数を定義
def log_likelihood(p):
    mp1 = p[0]
    sqecosw1 = p[1]
    sqesinw1 = p[2]
    wpM1 = p[3] # radian
    P1 = p[4]

    mp2 = p[5]
    sqecosw2 = p[6]
    sqesinw2 = p[7]
    wpM2 = p[8] # radian
    P2 = p[9]
    
    j1=p[10]
    j2=p[11]

    #あり得ない値を排除        
    if sqecosw1 <= -1. or sqecosw1 >= 1.0 or sqesinw1 <= -1. or sqesinw1 >= 1.0:
        return -np.inf
    
    if sqecosw2 <= -1. or sqecosw2 >= 1.0 or sqesinw2 <= -1. or sqesinw2 >= 1.0:
        return -np.inf
    
    e1 = sqecosw1**2 + sqesinw1**2
    e2 = sqecosw2**2 + sqesinw2**2

    if e1 <=1e-10 or e1 >= 0.9:
        return -np.inf

    if e2 <=1e-10 or e2 >= 0.9:
        return -np.inf
        
    if P1 < 0 or P2 < 0:
        return -np.inf
    
    if mp1 < 0:
        return -np.inf

    if mp2 < 0:
        return -np.inf

    if wpM1 < 0 or wpM1 > 2*np.pi: 
        return -np.inf
    
    if wpM2 < 0 or wpM2 > 2*np.pi:
        return -np.inf    
    
    if j1 <0 or j1>5 or j2<0 or j2>5:
        return-np.inf
    
  
    try:
        results = calc_ttvs_2pl(p)
    except:
        return -np.inf

    
    #配列データの整理
    pl = np.array(results['positions'][0])
    model_epoch = np.array(results['positions'][1])
    model_tt = np.array(results['positions'][2])
    model_vsky = np.array(results['positions'][4])

    pl1_index = np.where((pl==0) & (model_vsky>0))[0]
    pl2_index = np.where((pl==1) & (model_vsky>0))[0]

    model_epoch1 = model_epoch[pl1_index]
    model_epoch2 = model_epoch[pl2_index]
    model_tt1 = model_tt[pl1_index]
    model_tt2 = model_tt[pl2_index]

    #likelihoodの計算
    log_like=0   
    
    for i in range(len(epb)):
        Tc_model = model_tt1[model_epoch1==(int(epb[i]))]
        if(len(Tc_model)==0):
            return -np.inf
        
        if abs(ob_b[i] - Tc_model[0]) > 1: #[day]
            #print("bでエラー")
            return -np.inf
              
        log_like += (-0.5*(ob_b[i] - Tc_model[0])**2 / (ob_err_b[i]**2+j1**2))-0.5*np.log(2*np.pi*(ob_err_b[i]**2+j1**2))


    for i in range(len(epc)):
        Tc_model = model_tt2[model_epoch2==(int(epc[i]))]
        if(len(Tc_model)==0):
            return -np.inf

        if abs(ob_c[i] - Tc_model[0]) > 1:
            #print("cでエラー")
            return -np.inf
                         
        log_like += (-0.5*(ob_c[i] - Tc_model[0])**2 / (ob_err_c[i]**2+j2**2))-0.5*np.log(2*np.pi*(ob_err_c[i]**2+j2**2))
    return log_like    

def log_prior(P):
    lp=0
    return lp

def log_posterior(p):
    lp = log_prior(p)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(p)

def neg_log_posterior(p):
    return -1* log_posterior(p)

In [ ]:
def mcmc_first():
    mp1 = 3
    e1 = 0.05
    w1 = np.radians(-76)
    sqecosw1 = np.sqrt(e1)*np.cos(w1)
    sqesinw1 = np.sqrt(e1)*np.sin(w1)
    wpM1 = (Tstart-Tc0b)/Pb*2*np.pi + np.pi/2
    wpM1 = (wpM1/(2*np.pi) - np.floor(wpM1/(2*np.pi)))*2*np.pi #←上の議論はここからスタートしてる
    P1 =  6.398050683568169

    mp2 = 3
    e2 = 0.05
    w2 = np.radians(-41.6)
    sqecosw2 = np.sqrt(e2)*np.cos(w2)
    sqesinw2 = np.sqrt(e2)*np.sin(w2)
    wpM2 = (Tstart-Tc0c)/Pc*2*np.pi + np.pi/2
    wpM2 = (wpM2/(2*np.pi) - np.floor(wpM2/(2*np.pi)))*2*np.pi #0から2piに
    P2 = 18.87925686582526

    j1=0.001
    j2=0.001
    
    p0 = [mp1, sqecosw1, sqesinw1, wpM1, P1, mp2, sqecosw2, sqesinw2, wpM2, P2, j1, j2]

    print(log_posterior(p0))
    print(p0)
    
    #res = minimize(neg_log_posterior, p0, method='Nelder-Mead')
    #print(log_posterior(res.x))    
    #print(res.x)
    #mp1, sqecosw1, sqesinw1, wpM1, P1, mp2, sqecosw2, sqesinw2, wpM2, P2=res.x[0:10]
    
    e1=sqecosw1**2+sqesinw1**2
    e2=sqecosw2**2+sqesinw2**2
    
    #walkerを1000にしてcutする
    
    ndim, nwalkers = len(p0), 500

    mpmin1, mpmax1= mp1-0.1, mp1+0.1
    mpmin2, mpmax2= mp2-0.1, mp2+0.1

    #各walkerの初期位置をずらす
    pos = []
    for i in range(nwalkers):

        e1_tmp = e1 + 1e-7*np.random.rand(1)
        e2_tmp = e2 + 1e-7*np.random.rand(1)
        w1_tmp = np.pi*(2*np.random.rand(1)-1)
        w2_tmp = np.pi*(2*np.random.rand(1)-1)
        sqecosw1_tmp = np.sqrt(e1_tmp)*np.cos(w1_tmp)
        sqesinw1_tmp = np.sqrt(e1_tmp)*np.sin(w1_tmp)
        sqecosw2_tmp = np.sqrt(e2_tmp)*np.cos(w2_tmp)
        sqesinw2_tmp = np.sqrt(e2_tmp)*np.sin(w2_tmp)

        wpM1_tmp = (Tstart-Tc0b)/Pb*2*np.pi + np.pi/2. + 1e-5*np.random.randn(1)
        wpM1_tmp = (wpM1_tmp/(2*np.pi) - np.floor(wpM1_tmp/(2*np.pi)))*2*np.pi
        wpM2_tmp = (Tstart-Tc0c)/Pc*2*np.pi + np.pi/2. + 1e-5*np.random.randn(1)
        wpM2_tmp = (wpM2_tmp/(2*np.pi) - np.floor(wpM2_tmp/(2*np.pi)))*2*np.pi


        pos_tmp = np.empty(0)

        pos_tmp = np.append(pos_tmp, mpmin1 + (mpmax1-mpmin1)*np.random.rand(1)) # mp1
        pos_tmp = np.append(pos_tmp, np.array((sqecosw1_tmp,sqesinw1_tmp,wpM1_tmp)))  
        pos_tmp = np.append(pos_tmp, P1 + 1e-8*np.random.randn(1)) # P1

        pos_tmp = np.append(pos_tmp, mpmin2 + (mpmax2-mpmin2)*np.random.rand(1)) # mp2
        pos_tmp = np.append(pos_tmp, np.array((sqecosw2_tmp,sqesinw2_tmp,wpM2_tmp)))  
        pos_tmp = np.append(pos_tmp, P2 + 1e-8*np.random.randn(1)) # P2
        pos_tmp = np.append(pos_tmp, j1 + 1e-8*np.random.randn(1)) 
        pos_tmp = np.append(pos_tmp, j2 + 1e-8*np.random.randn(1)) 

        pos.append(pos_tmp)

    #stepの数を変更
    nsteps = 200000
    ndim = len(p0)

    filename = 'mcmc_ttvfit.hd5'
    #backend = emcee.backends.HDFBackend(filename)
    #backend.reset(nwalkers, ndim)

    with Pool(30) as pool:
        sampler = emcee.EnsembleSampler(nwalkers, ndim, log_posterior,moves=emcee.moves.DEMove(),pool=pool)
        sampler.run_mcmc(pos, nsteps, progress=True)

    #ここの値も変更
    discard=10000
    thin=10
    samples = sampler.get_chain(discard=discard, thin=thin, flat=True)

    for i in range(12):
        mcmc = np.percentile(samples[:, i], [16, 50, 84])
        q = np.diff(mcmc)
        print(mcmc[1], "+",q[1], "-", q[0])

    fig = corner.corner(samples, labels=["mb", "sqecosw1", "sqesinw1", "wpM1", "Pb", "mc", "sqecosw2", "sqesinw2", "wpM2", "Pc","j1","j2"],quantiles=[0.16,0.5,0.84],show_titles=True, title_fmt='.3f', label_kwargs={'fontsize':20})
    #plt.savefig("ttv_corner_try1_best_お試し.png", format="png", dpi=300)
    plt.show()

    log_prob = sampler.get_log_prob()
    print(log_prob)
    plt.figure(figsize=(8,4))
    plt.rcParams['font.size']=12
    for i in range(nwalkers):
        plt.plot(log_prob[:,i],alpha=0.1)
    plt.xlabel('Number of steps', fontsize=20)
    plt.ylabel('Log probability', fontsize=20)
    plt.show()

    log_prob = sampler.get_log_prob(flat=True, discard=discard, thin=thin)
    samples = sampler.get_chain(discard=discard, thin=thin, flat=True)
    argmax = np.argmax(log_prob)
    print(log_prob[argmax])
    print(samples[argmax,:])

    log_prob = sampler.get_log_prob()
    samples = sampler.get_chain()
    argsort = np.argsort(log_prob[-1])
    pos = samples[-1,:,:]
    
    return pos

In [ ]:
def mcmc_second(pos):
    #stepの数を変更
    nwalkers=500
    nsteps = 200000

    #配列のappendを理解
    #ここを解決

    filename = 'mcmc_ttvfit.hd5'
    #backend = emcee.backends.HDFBackend(filename)
    #backend.reset(nwalkers, ndim)

    #ここ
    with Pool(30) as pool:
        sampler = emcee.EnsembleSampler(nwalkers, 12, log_posterior,moves=emcee.moves.DEMove(), pool=pool)
        sampler.run_mcmc(pos, nsteps, progress=True)


    #ここの値も変更
    discard=10000
    thin=10
    samples = sampler.get_chain(discard=discard, thin=thin, flat=True)

    for i in range(12):
        mcmc = np.percentile(samples[:, i], [16, 50, 84])
        q = np.diff(mcmc)
        print(mcmc[1], "+",q[1], "-", q[0])

    fig = corner.corner(samples, labels=["mb", "sqecosw1", "sqesinw1", "wpM1", "Pb", "mc", "sqecosw2", "sqesinw2", "wpM2", "Pc","j1","j2"],quantiles=[0.16,0.5,0.84],show_titles=True, title_fmt='.3f', label_kwargs={'fontsize':20})
    #plt.savefig("ttv_corner_try1_best_お試し.png", format="png", dpi=300)
    plt.show()

    log_prob = sampler.get_log_prob()
    print(log_prob)
    plt.figure(figsize=(8,4))
    plt.rcParams['font.size']=12
    for i in range(nwalkers):
        plt.plot(log_prob[:,i],alpha=0.1)
    plt.xlabel('Number of steps', fontsize=20)
    plt.ylabel('Log probability', fontsize=20)
    plt.show()

    log_prob = sampler.get_log_prob(flat=True, discard=discard, thin=thin)
    samples = sampler.get_chain(discard=discard, thin=thin, flat=True)
    argmax = np.argmax(log_prob)
    print(log_prob[argmax])
    print(samples[argmax,:])

    log_prob = sampler.get_log_prob()
    samples = sampler.get_chain()
    argsort = np.argsort(log_prob[-1])
    pos = samples[-1,:,:]
    
    return pos
    #離心率が0.1程度であることが望ましい！

In [ ]:
def mcmc_third(pos):
    #stepの数を変更
    nwalkers=500
    nsteps = 200000

    #配列のappendを理解
    #ここを解決

    filename = 'mcmc_ttvfit.hd5'
    #backend = emcee.backends.HDFBackend(filename)
    #backend.reset(nwalkers, ndim)

    #ここ
    with Pool(30) as pool:
        sampler = emcee.EnsembleSampler(nwalkers, 12, log_posterior,moves=emcee.moves.DEMove(), pool=pool)
        sampler.run_mcmc(pos, nsteps, progress=True)


    #ここの値も変更
    discard=10000
    thin=10
    samples = sampler.get_chain(discard=discard, thin=thin, flat=True)

    for i in range(12):
        mcmc = np.percentile(samples[:, i], [16, 50, 84])
        q = np.diff(mcmc)
        print(mcmc[1], "+",q[1], "-", q[0])

    fig = corner.corner(samples, labels=["mb", "sqecosw1", "sqesinw1", "wpM1", "Pb", "mc", "sqecosw2", "sqesinw2", "wpM2", "Pc","j1","j2"],quantiles=[0.16,0.5,0.84],show_titles=True, title_fmt='.3f', label_kwargs={'fontsize':20})
    #plt.savefig("ttv_corner_try1_best_お試し.png", format="png", dpi=300)
    plt.show()

    log_prob = sampler.get_log_prob()
    print(log_prob)
    plt.figure(figsize=(8,4))
    plt.rcParams['font.size']=12
    for i in range(nwalkers):
        plt.plot(log_prob[:,i],alpha=0.1)
    plt.xlabel('Number of steps', fontsize=20)
    plt.ylabel('Log probability', fontsize=20)
    plt.show()

    log_prob = sampler.get_log_prob(flat=True, discard=discard, thin=thin)
    samples = sampler.get_chain(discard=discard, thin=thin, flat=True)
    argmax = np.argmax(log_prob)
    print(log_prob[argmax])
    print(samples[argmax,:])

    log_prob = sampler.get_log_prob()
    samples = sampler.get_chain()
    argsort = np.argsort(log_prob[-1])
    ntop=200
    index = argsort[-(ntop+1):-1]
    pos = samples[-1,index,:]
    
    return pos
    #離心率が0.1程度であることが望ましい！

In [ ]:
def mcmc(pos):
    nsteps=200000
    nwalkers=200
    ndim=len(pos[0])
    filename = 'mcmc_ttvfit.hd5'
    #backend = emcee.backends.HDFBackend(filename)
    #backend.reset(nwalkers, ndim)

    with Pool(30) as pool:
        sampler = emcee.EnsembleSampler(nwalkers, 12, log_posterior, pool=pool)
        sampler.run_mcmc(pos, nsteps, progress=True)

    #ここの値も変更
    discard=10000
    thin=50
    samples = sampler.get_chain(discard=discard, thin=thin, flat=True)

    for i in range(12):
        mcmc = np.percentile(samples[:, i], [16, 50, 84])
        q = np.diff(mcmc)
        print(mcmc[1], "+",q[1], "-", q[0])

    fig = corner.corner(samples, labels=["mb", "sqecosw1", "sqesinw1", "wpM1", "Pb", "mc", "sqecosw2", "sqesinw2", "wpM2", "Pc","j1","j2"],quantiles=[0.16,0.5,0.84],show_titles=True, title_fmt='.3f', label_kwargs={'fontsize':20})
    #plt.savefig("ttv_corner_try1_best_お試し.png", format="png", dpi=300)
    plt.show()

    log_prob = sampler.get_log_prob()
    print(log_prob)
    plt.figure(figsize=(8,4))
    plt.rcParams['font.size']=12
    for i in range(nwalkers):
        plt.plot(log_prob[:,i],alpha=0.1)
    plt.xlabel('Number of steps', fontsize=20)
    plt.ylabel('Log probability', fontsize=20)
    plt.show()

    log_prob = sampler.get_log_prob(flat=True, discard=discard, thin=thin)
    samples = sampler.get_chain(discard=discard, thin=thin, flat=True)
    argmax = np.argmax(log_prob)
    print(log_prob[argmax])
    print(samples[argmax,:])

    samples = sampler.get_chain()
    pos = samples[-1,:,:]
    return(pos)

In [ ]:
y=mcmc_first()

In [ ]:
y=mcmc_second(y)

In [ ]:
y=mcmc_third(y)

In [ ]:
y=mcmc(y)

In [ ]:
y=mcmc(y)

In [ ]:
y=mcmc(y)